# Multidimensional filtering assisted by SigmaCCS

In [1]:
import wget
import pandas as pd
from tqdm import tqdm
import numpy as np
import rdkit
from rdkit import Chem
from rdkit.Chem.Descriptors import ExactMolWt
from tqdm import *
from pandas import *
from matchms.importing import load_from_msp

## Download the LipidBlast dataset with negative ion mode

In [2]:
url = 'http://prime.psc.riken.jp/compms/msdial/download/lipidblast/MSDIAL-TandemMassSpectralAtlas-VS69-Neg.msp'
LipidBlastNeg = wget.download(url, ".")

100% [......................................................................] 418295847 / 418295847

## the removal of unpredictable adducts

In [3]:
LBneg_spectrum = list(load_from_msp(LipidBlastNeg))
ADDUCTS_LIST = ['[M+H]+','[M-H]-','[M+Na]+']
NameB, InchikeyB, AdductB, SMILESB, MZB = [], [], [], [], []
for i in tqdm(range(len(LBneg_spectrum))):
    if LBneg_spectrum[i].metadata['precursortype'] in ADDUCTS_LIST:
        NameB.append(LBneg_spectrum[i].metadata['compound_name'])
        InchikeyB.append(LBneg_spectrum[i].metadata['inchikey'])
        AdductB.append(LBneg_spectrum[i].metadata['precursortype'])
        SMILESB.append(LBneg_spectrum[i].metadata['smiles'])
        MZB.append(LBneg_spectrum[i].metadata['precursor_mz'])
data = {'NameB': NameB,
         'InchikeyB': InchikeyB,
         'AdductB': AdductB,
         'SMILESB': SMILESB,
         'MZB': MZB}
df = DataFrame(data)
df.to_excel('LiquidBlast_adduct_neg.xlsx', index=False)   

100%|███████████████████████████████████████████████████████████████████████| 792757/792757 [01:07<00:00, 11805.29it/s]


## Load the mouse lung dataset with 761 lipids in negative ion mode

In [4]:
lung_neg = pd.read_excel('Mouse_lung_adduct_negative.xlsx')  #query

## Taken the lipid (PubChem CID: 114944) as an example

### MList

In [5]:
neg_spectrum = df
tm = 0.00003
ln = lung_neg['MZ'][329]

metabolite, inchikey, adduct, smile, mz  = [], [], [], [], []
for i in tqdm(range(len(neg_spectrum))):
    dev = (abs(ln - neg_spectrum['MZB'][i]))/neg_spectrum['MZB'][i]
    if dev < tm:  
        metabolite.append(neg_spectrum['NameB'][i])
        inchikey.append(neg_spectrum['InchikeyB'][i])
        adduct.append(neg_spectrum['AdductB'][i])
        smile.append(neg_spectrum['SMILESB'][i])
        mz.append(neg_spectrum['MZB'][i])

data = {'NameB': metabolite,
         'InchikeyB': inchikey,
         'AdductB':adduct,
         'SMILESB': smile,
         'MZB': mz}

df = DataFrame(data)
df.to_excel('MList_329.xlsx', index=False)

100%|██████████████████████████████████████████████████████████████████████| 256696/256696 [00:01<00:00, 219271.29it/s]


### Load retention times of the molecules in the MList. Please refer to github GNN-RT repository for details of RT prediction

In [6]:
neg = pd.read_excel('MListWithPredictedRTs_329.xlsx')  #prediction

### RList

In [7]:
tr = 0.20

ln = lung_neg['RT'][329]

neg_rt = neg['Predict_RT']

metabolite, inchikey, adduct, smile, mz, rt  = [], [], [], [], [], []

for i in tqdm(range(len(neg_rt))):

    dev = (abs(ln - neg_rt[i]))/neg_rt[i]

    if dev < tr:
        metabolite.append(neg['NameB'][i])
        inchikey.append(neg['InchikeyB'][i])
        adduct.append(neg['AdductB'][i])
        smile.append(neg['SMILESB'][i])
        mz.append(neg['MZB'][i])
        rt.append(neg_rt[i])

data = {'NameB': metabolite,
         'InchikeyB': inchikey,
         'AdductB':adduct,
         'SMILESB': smile,
         'MZB': mz,
         'Predict_RT':rt}

df = DataFrame(data)
df.to_excel('RList_329.xlsx', index=False)

100%|███████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 79891.50it/s]


### Load the predicted CCS values of the molecules in the LipidBlast and then add CCS to the RList

In [8]:
allpredccs = pd.read_excel('LipidBlastPredCCS.xlsx')  
allpredccs_ccs = list(allpredccs['Pred_CCS'])
allpredccs_smi = list(allpredccs['SMILES'])

canccs = df
canccs_smi = list(canccs['SMILESB'])

canccs["SigmaCCS"] = ''

for i in tqdm(range(len(canccs_smi))):
    cal_mass = 0
    for j in range(len(allpredccs_smi)):
        if canccs_smi[i] == allpredccs_smi[j]:
            cal_mass = allpredccs_ccs[j]
            canccs.loc[i, 'SigmaCCS'] = cal_mass

canccs.to_excel('RListWithPredictedCCSs_329.xlsx', index=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 55/55 [00:00<00:00, 74.97it/s]


### CList

In [9]:
tc = 0.05

ln = lung_neg['CCS'][329]

neg = canccs
neg_ccs = neg['SigmaCCS']

metabolite, inchikey, adduct, smile, mz, rt, ccs  = [], [], [], [], [], [], []

for i in tqdm(range(len(neg_ccs))):

    dev = (abs(ln - neg_ccs[i]))/neg_ccs[i]

    if dev < tc:
        metabolite.append(neg['NameB'][i])
        inchikey.append(neg['InchikeyB'][i])
        adduct.append(neg['AdductB'][i])
        smile.append(neg['SMILESB'][i])
        mz.append(neg['MZB'][i])
        rt.append(neg['Predict_RT'][i])
        ccs.append(neg_ccs[i])

data = {'NameB': metabolite,
         'InchikeyB': inchikey,
         'AdductB':adduct,
         'SMILESB': smile,
         'MZB': mz,
         'Predict_RT':rt,
         'Predict_CCS' : ccs}

df = DataFrame(data)
df.to_excel('CList_329.xlsx', index=False)

100%|███████████████████████████████████████████████████████████████████████████████| 55/55 [00:00<00:00, 18324.47it/s]


### The fused scores in the CList

In [10]:
target_mols = lung_neg

label_CCS = list(target_mols['CCS'])[329]
label_RT = list(target_mols['RT'])[329]
label_mz = list(target_mols['MZ'])[329]

TOL_min_ccs = 0.015
TOL_max_ccs = 0.040
TOL_min_rt = 0.02
TOL_max_rt = 0.15
TOL_min_mz = 0.00002
TOL_max_mz = 0.00005

candidate = df
candidate["predict_mz"] = ''
smiles = list(candidate['SMILESB'])
add = list(target_mols['Adduct'])[329]
for s in range(len(candidate['SMILESB'])):
    smi = smiles[s]
    mol = Chem.MolFromSmiles(smi)
    cal_mass = ExactMolWt(mol)
    if add == '[M-H]-':
        cal_mass -= ExactMolWt(Chem.MolFromSmiles('[H+]'))
    candidate.loc[s, 'predict_mz'] = cal_mass
candidate['CCS_score'] = ''
candidate['RT_score'] = ''
candidate['mz_score'] = ''
pred_CCS = np.array(list(candidate['Predict_CCS']))
pred_RT = np.array(list(candidate['Predict_RT']))
pred_mz = np.array(list(candidate['predict_mz']))

relative_error = (abs(pred_CCS-label_CCS))/label_CCS
relative_error_rt = (abs(pred_RT-label_RT))/label_RT
relative_error_mz = (abs(pred_mz-label_mz))/label_mz


for j in range(len(relative_error)):
    if relative_error[j] < TOL_min_ccs:
        ccs_score = 1
    elif relative_error[j] > TOL_max_ccs:
        ccs_score = 0
    elif (relative_error[j] >= TOL_min_ccs) and (relative_error[j] <= TOL_max_ccs):
        ccs_score = 1-(relative_error[j] - TOL_min_ccs)/(TOL_max_ccs - TOL_min_ccs)
    candidate.loc[j,'CCS_score'] = ccs_score

for k in range(len(relative_error_rt)):
    if relative_error_rt[k] < TOL_min_rt:
        rt_score = 1
    elif relative_error_rt[k] > TOL_max_rt:
        rt_score = 0
    elif (relative_error_rt[k] >= TOL_min_rt) and (relative_error_rt[k] <= TOL_max_rt):
        rt_score = 1-(relative_error_rt[k] - TOL_min_rt)/(TOL_max_rt - TOL_min_rt)
    candidate.loc[k,'RT_score'] = rt_score


for l in range(len(relative_error_mz)):
    if relative_error_mz[l] < TOL_min_mz:
        mz_score = 1
    elif relative_error_mz[l] > TOL_max_mz:
        mz_score = 0
    elif (relative_error_mz[l] >= TOL_min_mz) and (relative_error_mz[l] <= TOL_max_mz):
        mz_score = 1-(relative_error_mz[l] - TOL_min_mz)/(TOL_max_mz - TOL_min_mz)
    candidate.loc[l,'mz_score'] = mz_score

for z in range(len(relative_error)):
    candidate.loc[z, 'score'] = candidate['CCS_score'][z] * 0.2 + candidate['RT_score'][z] * 0.2 + candidate['mz_score'][z] * 0.6

candidate.to_excel('CListwithFusedScore_329.xlsx', index=False) 